In [1]:
import numpy as np
import grid2op

from copy import deepcopy

from grid2op.dtypes import dt_int
from grid2op.dtypes import dt_bool
from grid2op.dtypes import dt_float

from grid2op.Agent import BaseAgent
from grid2op.Converter import IdToAct
from grid2op.PlotGrid import PlotMatplot
from grid2op.Runner import Runner

import warnings
warnings.filterwarnings("ignore")

In [2]:
import tensorflow as tf

In [3]:
env_name = "l2rpn_wcci_2020"
env = grid2op.make(env_name, test=True)
act = env.action_space()
obs = env.reset()
obs, reward, done, info = env.step(act)

print(f'act_type = {type(act)}''')
print(f'obs_type = {type(obs)}''')

print(f'rho = {len(obs.rho)}')
#print(f'line_status = {obs.line_status}')
#print(f'topo_vect = {len(obs.topo_vect)}')

print(f'action size = {act.size()}')
print(f'obs size = {obs.size()}')
print(f'reward = {reward}')
print(f'done = {done}')
print(f'info = {info}')
print(env.n_line)

act_type = <class 'grid2op.Space.GridObjects.ActionWCCI2020_l2rpn_wcci_2020'>
obs_type = <class 'grid2op.Space.GridObjects.ObservationWCCI2020_l2rpn_wcci_2020'>
rho = 59
action size = 494
obs size = 1266
reward = 89.17830657958984
done = False
info = {'disc_lines': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1], dtype=int32), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': False, 'is_illegal_reco': False, 'reason_alarm_illegal': None, 'opponent_attack_line': None, 'opponent_attack_sub': None, 'opponent_attack_duration': 0, 'exception': [], 'rewards': {}}
59


In [4]:
plot_helper = PlotMatplot(env.observation_space)
#_ = plot_helper.plot_info(gen_values=env.gen_pmax, line_values=env._thermal_limit_a, load_values=[el for el in range(env.n_load)])

In [8]:
class MyMLAgent(BaseAgent):
    
    def __init__(self, action_space):
        BaseAgent.__init__(self, action_space)
    
    def act(self, observation, reward, done=False):
        # action_space.n_line = len(observation.rho) = 59
        action = self.find_best_action(observation)
        return action
    
    def find_best_action(self, observation):
        self.tested_action = self._get_tested_action(observation)
        if len(self.tested_action) > 1:
            self.resulting_rewards = np.full(
                shape=len(self.tested_action), fill_value=np.NaN, dtype=dt_float
            )
            
            for i, action in enumerate(self.tested_action):
                (
                    simul_obs,
                    simul_reward,
                    simul_has_error,
                    simul_info,
                ) = observation.simulate(action)
                self.resulting_rewards[i] = simul_reward
            
            reward_idx = int(
                np.argmax(self.resulting_rewards)
            )  # rewards.index(max(rewards))
            best_action_sofar = self.tested_action[reward_idx]
            
        else:
            best_action_sofar = self.tested_action[0]
        
        best_action = self.find_best_line_to_reconnect(observation, best_action_sofar)
        #print(f'best_action = {best_action}')
        return best_action
    
    def find_best_line_to_reconnect(self, observation, original_action):
        # Fine all disconnected lines
        disconnected_lines = np.where(observation.line_status == False)[0]
        
        # Return the best action so far if no disconnected lines
        if not len(disconnected_lines):
            return original_action
        
        # Return the best action so far if those lines need timesteps to cooldown
        if (observation.time_before_cooldown_line[disconnected_lines] > 0).all(): # all: iterable or not.
            return original_action
        
        # Get rho of the best action so far 
        o, r, _, _ = obs.simulate(original_action)
        min_rho = o.rho.max()
        min_reward = r
        line_to_reconnect = -1
        
        # Iterate disconnected lines 
        for line in disconnected_lines:
            if not obs.time_before_cooldown_line[line]:
                reconnect_array = np.zeros_like(obs.rho, dtype=dt_int)
                reconnect_array[line] = 1
                reconnect_action = deepcopy(original_action)
                reconnect_action.update({'set_line_status': reconnect_array})
                
                # Simulate the reconnect action and find the minimum rho
                o, _, _, _ = obs.simulate(reconnect_action)
                if o.rho.max() < min_rho:
                    line_to_reconnect = line
                    min_rho = o.rho.max()
                    
            if line_to_reconnect != -1:
                reconnect_array = np.zeros_like(obs.rho, dtype=dt_int)
                reconnect_array[line_to_reconnect] = 1
                original_action.update({'set_line_status': reconnect_array})
                
            return original_action        
    '''
    def avoid_blackout(self, observation):
        threshold = 0.95
        if observation.rho.max() > threshold:
            l_id = np.where(observation.rho == observation.rho.max()[0])
            disconnect_line = self.action_space({"set_line_status": [(l_id, -1)]})
    '''
    def _get_tested_action(self, observation):
        res = [self.action_space({})]  # add the do nothing
        for i in range(self.action_space.n_line):
            for toggle_action in range(3): #Added this toggle to choose 3 actions
                tmp = np.full(self.action_space.n_line, fill_value=False, dtype=dt_bool)
                tmp[i] = True
                if toggle_action == 0:
                    action = self.action_space({"change_line_status": tmp})
                    if not observation.line_status[i]:
                        action = action.update(
                            {"set_bus": {"lines_or_id": [(i, 1)], "lines_ex_id": [(i, 1)]}})
                    res.append(action)
                elif toggle_action == 1:
                    if observation.line_status[i]:
                        action = self.action_space({"change_bus": {"lines_or_id": i}}) 
                        res.append(action)
                elif toggle_action == 2:
                    if observation.line_status[i]:
                        action = self.action_space({"change_bus": {"lines_ex_id": i}}) 
                        res.append(action)
        
        return res

In [9]:
my_agent = MyMLAgent(env.action_space)

runner = Runner(**env.get_params_for_runner(), agentClass=MyMLAgent)
res = runner.run(nb_episode=1, max_iter=3)

In [10]:
for _, chron_name, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics with id {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.6f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

	For chronics with id Scenario_april_000
		 - cumulative reward: 268.403198
		 - number of time steps completed: 3 / 3
